In [ ]:
import pandas as pd
import numpy as np
import xgboost as xg
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')

In [ ]:
columns = df_test.columns[1:]
X = df_train[columns].values
X_test = df_test[columns].values
target = df_train['loss'].values.reshape(-1,)

In [ ]:
train_oof = np.zeros((df_train.shape[0],))
test_preds = np.zeros((df_test.shape[0],))


In [ ]:
%%time
n_splits=10
kf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)

for i, (train_idx, val_idx) in enumerate(kf.split(df_train,target)):
    print("Fitting fold", i+1)
    train_features = X[train_idx]
    train_target = target[train_idx]

    
    val_features = X[val_idx]
    val_target = target[val_idx]
    
    model=xg.XGBRegressor(n_estimators = 1700,
                   max_depth  = 10,
                   learning_rate = 0.008,
                   reg_lambda = 0.1,
                   reg_alpha = 0.5,
                   booster = "gbtree",
                   tree_method = "gpu_hist",predictor = 'gpu_predictor',
                   subsample = 0.7,
                   colsample_bytree = 0.84,
                   n_jobs = 4,nthread = 4,
                   random_state = 123,
                )
    model.fit(train_features, train_target)
    val_pred = model.predict(val_features)
    train_oof[val_idx] = val_pred.flatten()
    test_preds += model.predict(X_test).flatten()/n_splits

In [ ]:
mean_squared_error(target,train_oof, squared=False)

In [ ]:
sample_submission['loss'] = test_preds
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()